## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.60,-73.97,57.00,87,86,1.23,overcast clouds
1,1,Kruisfontein,ZA,-34.00,24.73,63.97,64,98,6.08,overcast clouds
2,2,Puerto Ayora,EC,-0.74,-90.35,66.99,95,91,3.00,overcast clouds
3,3,Dikson,RU,73.51,80.55,20.03,96,61,10.54,broken clouds
4,4,Los Llanos De Aridane,ES,28.66,-17.92,68.00,60,0,8.05,clear sky


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
iten_min_temp = float(input("Please enter the minimum desired temperature for your vacation itenerary: "))
iten_max_temp = float(input("Please enter the maximum desired temperature for your vacation itenerary: "))

In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= iten_min_temp) & 
                                       (city_data_df['Max Temp'] <= iten_max_temp)]

In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                284
City                   284
Country                284
Lat                    284
Lng                    284
Max Temp               284
Humidity               284
Cloudiness             284
Wind Speed             284
Current Description    284
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Puerto Ayora,EC,66.99,overcast clouds,-0.74,-90.35,
4,Los Llanos De Aridane,ES,68.00,clear sky,28.66,-17.92,
7,New Norfolk,AU,66.99,overcast clouds,-42.78,147.06,
16,Hilo,US,82.40,clear sky,19.73,-155.09,
18,Vila Velha,BR,73.40,scattered clouds,-20.33,-40.29,
19,Saint-Philippe,RE,78.80,clear sky,-21.36,55.77,
22,Moerai,PF,71.85,overcast clouds,-22.43,-151.33,
23,Laguna,US,66.99,clear sky,38.42,-121.42,
25,Butaritari,KI,81.86,broken clouds,3.07,172.79,
26,Port Alfred,ZA,67.96,overcast clouds,-33.59,26.89,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        # Grab the first hotel from the response and store the name
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
        print(hotels['results'][0]['name'])
    except (IndexError):
        print('Hotel not found...skipping.')
        

Finch Bay Galapagos Hotel
Valle Aridane
The Shingles Riverside Cottages
Hilo Hawaiian Hotel
Hotel Vitória Palace
Chambres d'hôte "La Trinité"
Le Manotel
Holiday Inn Express & Suites Elk Grove Central - Hwy 99
Isles Sunset Lodge
The Halyards Hotel
Hotel Tesoro Los Cabos
Namatanai Hotel
Pension Maro'i
Observatory Guest House
Inubosaki Kanko Hotel
St Ives Hobart Accommodation Tasmania
Casa Dora, Bubaque
Dehing Guest House
Hotel not found...skipping.
Hotels in Lakshadweep Islands
کمپ خالدی
Bora-Bora Pearl Beach Resort & Spa
Villa Enata
Casa da Oscarina
Scoop Guest House
Mini Hotel Central
Shandrani Beachcomber Resort & Spa
Bils' Villa
Windvillage
Hotel Flamingo
Paradise Inn
Lorengau Harbourside Hotel
Shellharbour Resort & Conference Centre
Oumou Elghoura
BON Hotel Waterfront Richards Bay
Hotel Castelo
Best Western Plus Abbey Inn
Jati Hotel
Minas Hotel
Hotel Fairmont Makkah Clock Royal Tower
Nusa Island Retreat
Hotel Caribe
Lindsey Hôtel
Backpackers Inn on the Beach
Fortune Resort Bay Islan

In [25]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Puerto Ayora,EC,66.99,overcast clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
4,Los Llanos De Aridane,ES,68.00,clear sky,28.66,-17.92,Valle Aridane
7,New Norfolk,AU,66.99,overcast clouds,-42.78,147.06,The Shingles Riverside Cottages
16,Hilo,US,82.40,clear sky,19.73,-155.09,Hilo Hawaiian Hotel
18,Vila Velha,BR,73.40,scattered clouds,-20.33,-40.29,Hotel Vitória Palace
...,...,...,...,...,...,...,...
692,Codajas,BR,75.96,light rain,-3.84,-62.06,Hotel Tamires
696,Tirodi,IN,79.81,clear sky,21.68,79.70,Sameer Ali
697,Tahmek,MX,80.01,scattered clouds,20.88,-89.27,Hacienda San Jose Poniente
699,Rawson,AR,65.08,clear sky,-43.30,-65.10,Hotel Punta León


In [26]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))